### Import Dependencies ###

In [1]:
from fastmcp import Client

from pydantic import BaseModel, Field

from qdrant_client import QdrantClient
from qdrant_client.models import Prefetch, Filter, FieldCondition, MatchText, FusionQuery

from langsmith import traceable, get_current_run_tree

from langgraph.graph import StateGraph, START, END
from langgraph.prebuilt import ToolNode

from langchain_core.messages import AIMessage, ToolMessage, convert_to_openai_messages

from jinja2 import Template
from typing import Literal, Dict, Any, Annotated, List, Optional
from IPython.display import Image, display
from operator import add
from openai import OpenAI

from utils.utils import format_ai_message

import openai

import random
import ast
import inspect
import instructor
import json

### List available tools in MCP servers running on http://localhost:8001/mcp and http://localhost:8002/mcp ###

In [2]:
client = Client("http://localhost:8001/mcp")

In [4]:
async with client:
    tools  = await client.list_tools()

In [5]:
tools

[Tool(name='get_formatted_item_context', title=None, description='Get the top k context, each representing an inventory item for a given query.\n\nArgs:\n    query: The query to get the top k context for\n    top_k: The number of context chunks to retrieve, works best with 5 or more\n\nReturns:\n    A string of the top k context chunks with IDs and average ratings prepending each chunk, each representing an inventory item for a given query.', inputSchema={'properties': {'query': {'type': 'string'}, 'top_k': {'default': 5, 'type': 'integer'}}, 'required': ['query'], 'type': 'object'}, outputSchema={'properties': {'result': {'type': 'string'}}, 'required': ['result'], 'type': 'object', 'x-fastmcp-wrap-result': True}, icons=None, annotations=None, meta={'_fastmcp': {'tags': []}}, execution=None)]

In [7]:
print("======NAME=======")
print(tools[0].name)
print("======DESCRIPTION=======")
print(tools[0].description)
print("======INPUT SCHEMA=======")
print(tools[0].inputSchema)

======NAME=======
get_formatted_item_context
======DESCRIPTION=======
Get the top k context, each representing an inventory item for a given query.

Args:
    query: The query to get the top k context for
    top_k: The number of context chunks to retrieve, works best with 5 or more

Returns:
    A string of the top k context chunks with IDs and average ratings prepending each chunk, each representing an inventory item for a given query.
======INPUT SCHEMA=======
{'properties': {'query': {'type': 'string'}, 'top_k': {'default': 5, 'type': 'integer'}}, 'required': ['query'], 'type': 'object'}


In [12]:
client = Client("http://localhost:8002/mcp")

async with client:
    tools  = await client.list_tools()

In [13]:
tools


[Tool(name='get_formatted_reviews_context', title=None, description='Get the top k reviews matching a query for a list of prefiltered items.\n\nArgs:\n    query: The query to get the top k reviews for\n    item_list: The list of item IDs to prefilter for before running the query\n    top_k: The number of reviews to retrieve, this should be at least 20 if multipple items are prefiltered\n\nReturns:\n    A string of the top k context chunks with IDs prepending each chunk, each representing a review for a given inventory item for a given query.', inputSchema={'properties': {'query': {'type': 'string'}, 'item_list': {'items': {}, 'type': 'array'}, 'top_k': {'default': 15, 'type': 'integer'}}, 'required': ['query', 'item_list'], 'type': 'object'}, outputSchema={'properties': {'result': {'type': 'string'}}, 'required': ['result'], 'type': 'object', 'x-fastmcp-wrap-result': True}, icons=None, annotations=None, meta={'_fastmcp': {'tags': []}}, execution=None)]

In [14]:
print("======NAME=======")
print(tools[0].name)
print("======DESCRIPTION=======")
print(tools[0].description)
print("======INPUT SCHEMA=======")
print(tools[0].inputSchema)

======NAME=======
get_formatted_reviews_context
======DESCRIPTION=======
Get the top k reviews matching a query for a list of prefiltered items.

Args:
    query: The query to get the top k reviews for
    item_list: The list of item IDs to prefilter for before running the query
    top_k: The number of reviews to retrieve, this should be at least 20 if multipple items are prefiltered

Returns:
    A string of the top k context chunks with IDs prepending each chunk, each representing a review for a given inventory item for a given query.
======INPUT SCHEMA=======
{'properties': {'query': {'type': 'string'}, 'item_list': {'items': {}, 'type': 'array'}, 'top_k': {'default': 15, 'type': 'integer'}}, 'required': ['query', 'item_list'], 'type': 'object'}


### Execute a tool on one of the running MCP Servers ###

In [16]:
client = Client("http://localhost:8001/mcp")

async with client:
    result = await client.call_tool("get_formatted_items_context", {"query": "What earphones can I get?", "top_k": 5})

ToolError: Unknown tool: get_formatted_items_context